### Imports

In [ ]:
%pip install seaborn sklearn keras_tuner lightgbm plotly &> /dev/null

In [ ]:
# To store data
# Logging
import logging
import warnings

import matplotlib.pyplot as plt

# To do linear algebra
import numpy as np

# To get new datatypes and functions
from cycler import cycler
from matplotlib.cm import get_cmap

# To create plots
from matplotlib.colors import rgb2hex
from numpy import pi
from scipy.optimize import curve_fit

import pandas as pd
import plotly.graph_objs as go

# To create nicer plots
import seaborn as sns
from keras.layers import Dense, Dropout

# To build models
from keras.models import Sequential
from keras_tuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from kerastuner.tuners import RandomSearch

# To gbm light
from lightgbm import LGBMClassifier

# To create interactive plots
from plotly.offline import init_notebook_mode, iplot

# To process data
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow import keras
from tensorflow.keras import layers

# To investigate distributions
# from scipy.stats import norm, probplot, skew
# from sklearn.decomposition import PCA
# from sklearn.manifold import TSNE
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler

logging.getLogger("tensorflow").setLevel(logging.ERROR)
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)


### Uncompressing data

In [ ]:
!./unzip.sh test.csv.zip train.csv.zip

In [ ]:
# Load datasets
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

# Combine boths dataframes
train_data["Data"] = "Train"
test_data["Data"] = "Test"
both_data = pd.concat([train_data, test_data], axis=0).reset_index(drop=True)
both_data["subject"] = "#" + both_data["subject"].astype(str)

# Create label
label = both_data.pop("Activity")

print("Shape Train:\t{}".format(train_data.shape))
print("Shape Test:\t{}\n".format(test_data.shape))

train_data.head()


### Exploratory Data Analysis

In [ ]:
train_data.shape


In [ ]:
train_data.head()


In [ ]:
train_data.describe()


### Checking categorical imbalance

In [ ]:
import plotly.express as px

# dummy change
fig = px.pie(train_data, names="Activity", width=700)
fig.update_layout(
    title={
        "text": "Activities distribution in the data",
        "y": 0.95,
        "x": 0.45,
        "xanchor": "center",
        "yanchor": "top",
    }
)
fig.show()


In [ ]:
# Plotting data
label_counts = label.value_counts()

# Get colors
n = label_counts.shape[0]
colormap = get_cmap("viridis")
colors = [rgb2hex(colormap(col)) for col in np.arange(0, 1.01, 1 / (n - 1))]

# Create plot
data = go.Bar(x=label_counts.index, y=label_counts, marker=dict(color=colors))

layout = go.Layout(
    title="Smartphone Activity Label Distribution",
    xaxis=dict(title="Activity"),
    yaxis=dict(title="Count"),
)

fig = go.Figure(data=[data], layout=layout)
iplot(fig)


## Prepare Train And Test Data

In [ ]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
x_train, y_train = train_data.iloc[:, :-2], train_data.iloc[:, -1:]
x_test, y_test = test_data.iloc[:, :-2], test_data.iloc[:, -1:]
x_train.shape, y_train.shape


In [ ]:
x_test, y_test = test_data.iloc[:, :-2], test_data.iloc[:, -1:]
x_test.shape, y_test.shape


### One-hot Encoding

In [ ]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)


In [ ]:
x_test.shape, y_test.shape, x_train.shape, y_train.shape


### Scaling

In [ ]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)


## Base Model

In [ ]:
model = Sequential()
model.add(
    Dense(
        units=64,
        kernel_initializer="normal",
        activation="sigmoid",
        input_dim=x_train.shape[1],
    )
)
model.add(Dropout(0.2))
model.add(Dense(units=6, kernel_initializer="normal", activation="softmax"))
model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
history = model.fit(
    x_train, y_train, batch_size=64, epochs=10, validation_data=(x_test, y_test)
)
model.summary()


## Tuning the model

In [ ]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int("num_layers", 2, 25)):
        model.add(
            layers.Dense(
                units=hp.Int("units" + str(i), min_value=32, max_value=512, step=32),
                kernel_initializer=hp.Choice("initializer", ["uniform", "normal"]),
                activation=hp.Choice("activation", ["relu", "sigmoid", "tanh"]),
            )
        )
    model.add(
        layers.Dense(
            6,
            kernel_initializer=hp.Choice("initializer", ["uniform", "normal"]),
            activation="softmax",
        )
    )
    model.add(Dropout(0.2))
    model.compile(
        optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )
    return model


tuner = RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=5,
    executions_per_trial=3,
    directory="project",
    project_name="Human_activity_recognition",
)

tuner.search_space_summary()


In [ ]:
tuner.search(x_train, y_train, epochs=10, validation_data=(x_test, y_test))


In [ ]:
tuner.results_summary()


In [ ]:
model = tuner.get_best_models(num_models=1)[0]
history = model.fit(x_train, y_train, epochs=51, validation_data=(x_test, y_test))


In [ ]:
model.summary()

import tensorflow as tf
from tensorflow import keras

Callback = tf.keras.callbacks.EarlyStopping(monitor="accuracy", patience=3)
mo_fitt = model.fit(
    x_train, y_train, epochs=200, validation_data=(x_test, y_test), callbacks=Callback
)


In [ ]:
accuracy = mo_fitt.history["accuracy"]
loss = mo_fitt.history["loss"]
validation_loss = mo_fitt.history["val_loss"]
validation_accuracy = mo_fitt.history["val_accuracy"]

# dynamically set x based on y
x = range(len(validation_loss))

plt.figure(figsize=(15, 7))

plt.subplot(2, 2, 1)
plt.plot(x, accuracy, label="Training Accuracy", color="blue", linewidth=2)
plt.plot(
    x, validation_accuracy, label="Validation Accuracy", color="green", linewidth=2
)
plt.legend(loc="lower right", fontsize=14)
plt.title("Accuracy: Training vs Validation", fontsize=16)
plt.xlabel("Epoch", fontsize=14)
plt.ylabel("Accuracy", fontsize=14)
plt.grid(True)

plt.subplot(2, 2, 2)
plt.plot(x, loss, label="Training Loss", color="blue", linewidth=2)
plt.plot(x, validation_loss, label="Validation Loss", color="green", linewidth=2)
plt.legend(loc="upper right", fontsize=14)
plt.title("Loss: Training vs Validation", fontsize=16)
plt.xlabel("Epoch", fontsize=14)
plt.ylabel("Loss", fontsize=14)
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# Save model(s) for development purposes
import tensorflow as tf

# create a TFLiteConverter object
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# convert the model to TFLite format
tflite_model = converter.convert()

# save the TFLite model to a file
with open("./assets/model.tflite", "wb") as f:
    f.write(tflite_model)
